**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
from os.path import join

from keras.models import Sequential, model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D, Reshape, BatchNormalization, Flatten

VIDEO_FOLDER = './videos'
PRETRAINED_FOLDER = './saved_models'

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

> During inference time, `act` returns the action following the policy learned by the agent. During training, the agent only do so with a probability of $1 - \epsilon$. With a probability of $\epsilon$, the agent takes a random action.
The reason for this is to have the model "explore" other possible actions than just the one corresponding to the policy. If $\epsilon$ is set to 0, the agent will almost surely get stuck on a sub-optimal policy - it will only follows the policy learned in the first few steps of the training without trying out other possible actions. $\epsilon$ sets the trade-off between exploration and exploitation.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(join(VIDEO_FOLDER, str(e) + '.mp4'), self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T = 200
temperature = 0.3
epochs_train = 5 # set small when debugging
epochs_test = 5 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(join(VIDEO_FOLDER, name), 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

> ```position``` and ```board``` have both a size of $(GridSize + 4)^2$.  While the mouse can only move in a grid with a size of $GridSize$, this trick is used to return a state of consistent size ($5\times5$) even when the mouse is on an edge of the board. 

> ```board``` contains the position of the cheese and poisonous cells, more precisely, the value of the corresponding reward. ```position``` contains the boundaries of the grid of the size $GridSize$ : the mouse can't go to the cells whose values are $-1$.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        """Return random action"""
        return np.random.randint(self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
    for e in range(epochs):
        state = env.reset()
        game_over = False
        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win - lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+ str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T, temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent, env, epochs_test, prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 8.0/18.0. Average score (-10.0)
Win/lose count 11.0/8.0. Average score (-3.5)
Win/lose count 9.5/13.0. Average score (-3.5)
Win/lose count 13.0/11.0. Average score (-2.125)
Win/lose count 16.0/13.0. Average score (-1.1)
Final score: -1.1


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**1.**
>  \begin{align*}
    Q^\pi(s,a) &= E_{p^{\pi}}(\sum_{t \geq 0}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a)\\
               &= E_{p^{\pi}}(r(s, a) + \gamma \sum_{t \geq 0}\gamma^{t}r(s_{t+1},a_{t+1}))\\
               &= r(s, a) + \gamma \sum_{s'_1, a'_1} [p(s_1=s'_1, a_1=a'_1 | s_0=s, a_0=a) E_{p^{\pi}}(\sum_{t \geq 0}\gamma^{t}r(s_{t+1},a_{t+1}) | s_1=s'_1, a_1=a'_1)]\\
               &= r(s, a) + \gamma \sum_{s'_1, a'_1} p(s_1=s'_1, a_1=a'_1 | s_0=s, a_0=a) Q(s'_1, a'_1)\\
    &= r(s, a) + \gamma E_{s', a' \sim p(.|s, a)}[Q^\pi(s', a')]
  \end{align*}



**2.**
>  $$Q^*(s, a) = \max_\pi Q^\pi(s,a)$$
 
 We can derive from the previous result that:

>  \begin{align*}
    Q^*(s,a) &= \max_\pi(r(s, a) + \gamma \sum_{s'_1, a'_1} p(s_1=s'_1, a_1=a'_1 | s_0=s, a_0=a) Q^\pi(s'_1, a'_1))\\
             &= \max_\pi(r(s, a) + \gamma \sum_{s'_1} p(s_1=s'_1 | s_0=s, a_0=a) Q^\pi(s'_1, \pi(s'_1)))\\
             &= r(s, a) + \gamma \sum_{s'_1} p(s_1=s'_1 | s_0=s, a_0=a) \max_\pi(Q^\pi(s'_1, \pi(s'_1)))\\
             &= r(s, a) + \gamma \sum_{s'_1} p(s_1=s'_1 | s_0=s, a_0=a) Q^*(s'_1, \pi(s'_1)))
  \end{align*}

  Using the fact that $Q^*(s', \pi(s') = \max_{a'} Q^*(s', a')$, we finally get

>  \begin{align*}
    Q^*(s,a) &= r(s, a) + \gamma \sum_{s'_1} p(s_1=s'_1 | s_0=s, a_0=a) \max_{a'} Q^*(s'_1, a'))\\
             &= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]
  \end{align*}



**3.** 

>\begin{align*}
E_{s' \sim \pi^*(.|s,a)} [r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)] &= Q^*(s,a,\theta)-Q(s,a,\theta)
\end{align*}

>according to the second formula. Minizing this distance corresponds to getting $\theta$ so that $Q$ corresponds to the optimal policy. If it converges, using $Q$ to choose the action to take will lead the agent to follow the optimal policy.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
import random

class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            self.memory.pop(0)

    def random_access(self):
         return random.choice(self.memory)

***
The pipeline we will use for training is given below:

In [10]:
def train(agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward,
                                   game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix + str(e))

        # Update stats
        score += win - lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})".
              format(e, epoch, loss, win, lose, win - lose))

        agent.save(name_weights=prefix + 'model_e_{}.h5'.format(e), name_model=prefix + 'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self,
                 grid_size=13,
                 epsilon=0.1,
                 memory_size=100,
                 batch_size=16,
                 n_state=2):
        super(DQN, self).__init__(epsilon=epsilon)

        # Discount for Q learning
        self.discount = 0.99

        self.grid_size = grid_size

        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)

        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.array([s]))[0])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])

        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))

        next_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        actions = np.zeros(self.batch_size, dtype=int)
        rewards = np.zeros(self.batch_size)
        game_overs = np.zeros(self.batch_size)

        for i in range(self.batch_size):
            input_states[i], next_states[i], actions[i], rewards[
                i], game_overs[i] = self.memory.random_access()

        Q_current = self.model.predict(input_states)
        Q_next = self.model.predict(next_states)
        target_q = Q_current.copy()

        for i, (game_over, action, reward, Q_n) in enumerate(
                zip(game_overs, actions, rewards, Q_next)):
            if game_over:
                target = reward
            else:
                target = reward + self.discount * np.max(Q_n)
            target_q[i, action] = target

        # Clip to avoid exploding gradient
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self, name_weights='model.h5', name_model='model.json'):
        self.model.save_weights(join(PRETRAINED_FOLDER, name_weights), overwrite=True)
        with open(join(PRETRAINED_FOLDER, name_model), "w") as outfile:
            json.dump(self.model.to_json(), outfile)

    def load(self, name_weights='model.h5', name_model='model.json'):
        with open(join(PRETRAINED_FOLDER, name_model), "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(join(PRETRAINED_FOLDER, name_weights))
        model.compile("sgd", "mse")
        self.model = model


class DQN_FC(DQN):
    def __init__(self, lr=0.1, **kwargs):
        super(DQN_FC, self).__init__(**kwargs)

        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        model.add(Dense(48, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.n_action, activation='linear'))

        model.compile(sgd(lr=lr, decay=1e-4, momentum=0), "mse")
        self.model = model

In [12]:
epochs_train = 200

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(grid_size=size, lr=.1, epsilon=0.1, memory_size=2000, batch_size=64)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train190.mp4'))

Epoch 000/200 | Loss 0.0107 | Win/lose count 1.5/1.0 (0.5)
Epoch 001/200 | Loss 0.0259 | Win/lose count 3.5/1.0 (2.5)
Epoch 002/200 | Loss 0.0057 | Win/lose count 2.5/4.0 (-1.5)
Epoch 003/200 | Loss 0.0314 | Win/lose count 2.0/1.0 (1.0)
Epoch 004/200 | Loss 0.0072 | Win/lose count 1.5/4.0 (-2.5)
Epoch 005/200 | Loss 0.0041 | Win/lose count 2.5/5.0 (-2.5)
Epoch 006/200 | Loss 0.0074 | Win/lose count 3.0/2.0 (1.0)
Epoch 007/200 | Loss 0.0058 | Win/lose count 6.0/4.0 (2.0)
Epoch 008/200 | Loss 0.0059 | Win/lose count 1.5/5.0 (-3.5)
Epoch 009/200 | Loss 0.0217 | Win/lose count 3.0/6.0 (-3.0)
Epoch 010/200 | Loss 0.0057 | Win/lose count 4.0/2.0 (2.0)
Epoch 011/200 | Loss 0.0029 | Win/lose count 2.0/1.0 (1.0)
Epoch 012/200 | Loss 0.0198 | Win/lose count 3.5/7.0 (-3.5)
Epoch 013/200 | Loss 0.0062 | Win/lose count 4.5/5.0 (-0.5)
Epoch 014/200 | Loss 0.0043 | Win/lose count 5.0/7.0 (-2.0)
Epoch 015/200 | Loss 0.0107 | Win/lose count 4.0/3.0 (1.0)
Epoch 016/200 | Loss 0.0090 | Win/lose count 5.5

Epoch 138/200 | Loss 0.0050 | Win/lose count 3.0/0 (3.0)
Epoch 139/200 | Loss 0.0022 | Win/lose count 26.0/0 (26.0)
Epoch 140/200 | Loss 0.0035 | Win/lose count 13.5/1.0 (12.5)
Epoch 141/200 | Loss 0.0046 | Win/lose count 10.5/3.0 (7.5)
Epoch 142/200 | Loss 0.0459 | Win/lose count 6.0/0 (6.0)
Epoch 143/200 | Loss 0.0028 | Win/lose count 13.5/3.0 (10.5)
Epoch 144/200 | Loss 0.0037 | Win/lose count 14.5/1.0 (13.5)
Epoch 145/200 | Loss 0.0047 | Win/lose count 12.5/3.0 (9.5)
Epoch 146/200 | Loss 0.0052 | Win/lose count 11.5/1.0 (10.5)
Epoch 147/200 | Loss 0.0297 | Win/lose count 17.5/2.0 (15.5)
Epoch 148/200 | Loss 0.0033 | Win/lose count 12.0/5.0 (7.0)
Epoch 149/200 | Loss 0.0028 | Win/lose count 14.5/0 (14.5)
Epoch 150/200 | Loss 0.0036 | Win/lose count 8.5/3.0 (5.5)
Epoch 151/200 | Loss 0.0047 | Win/lose count 10.5/1.0 (9.5)
Epoch 152/200 | Loss 0.0293 | Win/lose count 5.5/3.0 (2.5)
Epoch 153/200 | Loss 0.0059 | Win/lose count 7.5/0 (7.5)
Epoch 154/200 | Loss 0.0049 | Win/lose count 11.

In [13]:
agent.save(name_weights='fc_trainmodel_e_final.h5',name_model='fc_trainmodel.json')

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [14]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(16, (2,2), activation = 'relu'))
        model.add(Conv2D(16, (2,2), activation = 'relu'))
        model.add(Flatten())
        model.add(Dense(self.n_action))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [15]:
epochs_train=200

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(grid_size=size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train190.mp4'))

Epoch 000/200 | Loss 0.0050 | Win/lose count 4.0/7.0 (-3.0)
Epoch 001/200 | Loss 0.0070 | Win/lose count 5.0/3.0 (2.0)
Epoch 002/200 | Loss 0.0061 | Win/lose count 3.0/1.0 (2.0)
Epoch 003/200 | Loss 0.0451 | Win/lose count 2.5/4.0 (-1.5)
Epoch 004/200 | Loss 0.0052 | Win/lose count 3.5/2.0 (1.5)
Epoch 005/200 | Loss 0.0013 | Win/lose count 3.0/3.0 (0.0)
Epoch 006/200 | Loss 0.0274 | Win/lose count 4.5/4.0 (0.5)
Epoch 007/200 | Loss 0.0614 | Win/lose count 6.5/1.0 (5.5)
Epoch 008/200 | Loss 0.0017 | Win/lose count 5.0/4.0 (1.0)
Epoch 009/200 | Loss 0.0088 | Win/lose count 3.5/4.0 (-0.5)
Epoch 010/200 | Loss 0.0414 | Win/lose count 4.5/2.0 (2.5)
Epoch 011/200 | Loss 0.0024 | Win/lose count 5.5/4.0 (1.5)
Epoch 012/200 | Loss 0.0072 | Win/lose count 6.0/2.0 (4.0)
Epoch 013/200 | Loss 0.0016 | Win/lose count 6.5/1.0 (5.5)
Epoch 014/200 | Loss 0.0034 | Win/lose count 3.0/3.0 (0.0)
Epoch 015/200 | Loss 0.0022 | Win/lose count 3.0/2.0 (1.0)
Epoch 016/200 | Loss 0.0557 | Win/lose count 12.5/7.0

Epoch 137/200 | Loss 0.0021 | Win/lose count 16.5/2.0 (14.5)
Epoch 138/200 | Loss 0.0033 | Win/lose count 20.5/5.0 (15.5)
Epoch 139/200 | Loss 0.0577 | Win/lose count 14.5/6.0 (8.5)
Epoch 140/200 | Loss 0.0027 | Win/lose count 20.0/1.0 (19.0)
Epoch 141/200 | Loss 0.0015 | Win/lose count 13.0/3.0 (10.0)
Epoch 142/200 | Loss 0.0032 | Win/lose count 22.5/5.0 (17.5)
Epoch 143/200 | Loss 0.0100 | Win/lose count 20.5/5.0 (15.5)
Epoch 144/200 | Loss 0.0513 | Win/lose count 23.0/9.0 (14.0)
Epoch 145/200 | Loss 0.0028 | Win/lose count 11.0/3.0 (8.0)
Epoch 146/200 | Loss 0.0037 | Win/lose count 22.0/4.0 (18.0)
Epoch 147/200 | Loss 0.0030 | Win/lose count 19.0/3.0 (16.0)
Epoch 148/200 | Loss 0.0034 | Win/lose count 11.5/3.0 (8.5)
Epoch 149/200 | Loss 0.0039 | Win/lose count 18.5/2.0 (16.5)
Epoch 150/200 | Loss 0.0031 | Win/lose count 20.0/3.0 (17.0)
Epoch 151/200 | Loss 0.0021 | Win/lose count 18.5/3.0 (15.5)
Epoch 152/200 | Loss 0.0026 | Win/lose count 15.5/5.0 (10.5)
Epoch 153/200 | Loss 0.0517

In [16]:
HTML(display_videos('cnn_train190.mp4'))

In [17]:
agent.save(name_weights='cnn_trainmodel_e_final.h5',name_model='cnn_trainmodel.json')

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [18]:
epochs_test = 15

env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(grid_size=size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel_e_final.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(grid_size=size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel_e_final.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 9.0/1.0. Average score (8.0)
Win/lose count 14.0/3.0. Average score (9.5)
Win/lose count 14.5/2.0. Average score (10.5)
Win/lose count 14.5/0. Average score (11.5)
Win/lose count 15.0/4.0. Average score (11.4)
Win/lose count 12.0/4.0. Average score (10.833333333333334)
Win/lose count 11.5/5.0. Average score (10.214285714285714)
Win/lose count 14.5/5.0. Average score (10.125)
Win/lose count 15.5/3.0. Average score (10.38888888888889)
Win/lose count 11.5/2.0. Average score (10.3)
Win/lose count 22.5/6.0. Average score (10.863636363636363)
Win/lose count 20.0/2.0. Average score (11.458333333333334)
Win/lose count 12.5/4.0. Average score (11.23076923076923)
Win/lose count 11.0/4.0. Average score (10.928571428571429)
Win/lose count 16.5/1.0. Average score (11.233333333333333)
Final score: 11.233333333333333
Test of the FC
Win/lose count 20.5/1.0. Average score (19.5)
Win/lose count 17.5/2.0. Average score (17.5)
Win/lose count 14.5/3.0. Average score (15.5)
Wi

In [19]:
HTML(display_videos('cnn_test10.mp4'))

In [20]:
HTML(display_videos('fc_test10.mp4'))

> The temperature parameter controls the amount of rewards/maluses on the board. Given the implementation of environment, the great the temperature T is, the more rewards on the board there will be. (The probability of a reward on a given cell of the board is $P(\text{ Reward }) = T$, while $P(\text{ Poison }) = T(1-T)$). We can then expect the scores to be greater with a larger value of temperature.

> Both agents perform rather well. The DNN-based agents performs slightly better than the CNN one.

> The current agents tends to get stuck in corners : if they can't see any rewards around, they will simply stay in a corner until the game ends while there are probably other rewards to collect on the board.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [30]:
def exponential_decrease(step):
    return np.exp(-step)

def train_explore(agent, env, epoch, epsilon_func=exponential_decrease, prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        step = 0

        while not game_over:
            # Decrease epsilon
            agent.set_epsilon(epsilon_func(step))
            step += 1
            
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward,
                                   game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix + str(e))

        # Update stats
        score += win - lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.0f}/{:.0f} ({:.0f})".
              format(e, epoch, loss, win, lose, win - lose))
        agent.save(
              name_weights=prefix + 'model.h5', name_model=prefix + 'model.json')


class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size, grid_size))
        self.position = np.zeros((grid_size, grid_size))
        self.malus_position = np.zeros((grid_size, grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(join(VIDEO_FOLDER, str(e) + '.mp4'), self.to_draw)


    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        
        if train:
             reward = -self.malus_position[self.x, self.y]
        else:
            reward = 0
        self.malus_position[self.x, self.y] = min(self.malus_position[self.x, self.y] + 0.05, 0.2)

        self.t = self.t + 1
        reward += self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        # 3 "feature" state
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [31]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/200 | Loss 0.0109 | Win/lose count 4/35 (-31)
Epoch 001/200 | Loss 0.0306 | Win/lose count 2/38 (-36)
Epoch 002/200 | Loss 0.0379 | Win/lose count 7/34 (-27)
Epoch 003/200 | Loss 0.0055 | Win/lose count 8/36 (-28)
Epoch 004/200 | Loss 0.0067 | Win/lose count 3/36 (-33)
Epoch 005/200 | Loss 0.0079 | Win/lose count 8/35 (-26)
Epoch 006/200 | Loss 0.0095 | Win/lose count 1/42 (-42)
Epoch 007/200 | Loss 0.0304 | Win/lose count 12/27 (-15)
Epoch 008/200 | Loss 0.0049 | Win/lose count 5/35 (-29)
Epoch 009/200 | Loss 0.0278 | Win/lose count 6/38 (-33)
Epoch 010/200 | Loss 0.0072 | Win/lose count 4/40 (-35)
Epoch 011/200 | Loss 0.0062 | Win/lose count 2/41 (-38)
Epoch 012/200 | Loss 0.0366 | Win/lose count 6/41 (-35)
Epoch 013/200 | Loss 0.0353 | Win/lose count 8/39 (-32)
Epoch 014/200 | Loss 0.0046 | Win/lose count 6/40 (-34)
Epoch 015/200 | Loss 0.0049 | Win/lose count 4/40 (-36)
Epoch 016/200 | Loss 0.0078 | Win/lose count 8/41 (-34)
Epoch 017/200 | Loss 0.0321 | Win/lose count 8/

Epoch 145/200 | Loss 0.0100 | Win/lose count 14/38 (-24)
Epoch 146/200 | Loss 0.0054 | Win/lose count 6/40 (-34)
Epoch 147/200 | Loss 0.0089 | Win/lose count 13/38 (-25)
Epoch 148/200 | Loss 0.0080 | Win/lose count 16/33 (-17)
Epoch 149/200 | Loss 0.0092 | Win/lose count 11/37 (-27)
Epoch 150/200 | Loss 0.0789 | Win/lose count 7/42 (-35)
Epoch 151/200 | Loss 0.0059 | Win/lose count 15/32 (-18)
Epoch 152/200 | Loss 0.0088 | Win/lose count 7/38 (-31)
Epoch 153/200 | Loss 0.0124 | Win/lose count 15/36 (-21)
Epoch 154/200 | Loss 0.0059 | Win/lose count 14/33 (-19)
Epoch 155/200 | Loss 0.0077 | Win/lose count 8/43 (-34)
Epoch 156/200 | Loss 0.0352 | Win/lose count 16/32 (-16)
Epoch 157/200 | Loss 0.0108 | Win/lose count 11/34 (-23)
Epoch 158/200 | Loss 0.0086 | Win/lose count 10/36 (-26)
Epoch 159/200 | Loss 0.0092 | Win/lose count 14/35 (-21)
Epoch 160/200 | Loss 0.0396 | Win/lose count 11/39 (-28)
Epoch 161/200 | Loss 0.0144 | Win/lose count 14/44 (-30)
Epoch 162/200 | Loss 0.0078 | Win/l

In [32]:
# Evaluation
epochs_test = 11

env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 3.5/7.0. Average score (-3.5)
Win/lose count 4.0/1.0. Average score (-0.25)
Win/lose count 6.5/3.0. Average score (1.0)
Win/lose count 3.5/2.0. Average score (1.125)
Win/lose count 1.5/0. Average score (1.2)
Win/lose count 8.0/3.0. Average score (1.8333333333333333)
Win/lose count 6.5/3.0. Average score (2.0714285714285716)
Win/lose count 4.0/4.0. Average score (1.8125)
Win/lose count 2.5/1.0. Average score (1.7777777777777777)
Win/lose count 8.5/3.0. Average score (2.15)
Win/lose count 6.0/2.0. Average score (2.3181818181818183)
Final score: 2.3181818181818183


In [24]:
HTML(display_videos('cnn_train_explore160.mp4'))

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***